# Self Ask with Search

In [1]:
from langchain import hub
from langchain.agents import AgentExecutor, create_self_ask_with_search_agent

from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain.agents import load_tools
from langchain.tools import BaseTool, StructuredTool, tool
from langchain import PromptTemplate
from langchain.schema import StrOutputParser

In [2]:
llm = OpenAI()

In [3]:
prompt = hub.pull("hwchase17/self-ask-with-search")

/Users/hyeonjinho/.pyenv/versions/3.11.6/lib/python3.11/site-packages/langsmith/client.py:278: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [4]:
print(prompt.template)

Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washin

In [5]:
search = load_tools(["ddg-search"])[0]

In [6]:
search.run("Where is Martin Campbell from?")

'Early Life and Family Background. Martin Campbell was born on October 24, 1943, in Hastings, East Sussex, England. Growing up in a working-class family, Campbell\'s early life was marked by modest beginnings, yet it was clear from a young age that he had a passion for storytelling and the arts. His family\'s support, though humble, played a pivotal role in encouraging his artistic ambitions. EXCLUSIVE: Former James Bond franchise director Martin Campbell speaks on whether any past candidate should be considered for the 007 role in Bond 26. The search for a new Bond continues. Screen Rant Daniel Craig and Martin Campbell during Daniel Craig Unveiled as the New James Bond in "Casino Royale" - Photocall at HMS President 72, St Katharine\'s Way E1 in London, Great Britain. Martin Campbell became a sought-after action director thanks to his work on Goldeneye, the 1995 Pierce Brosnan Bond movie, which grossed $356 million worldwide. Campbell would later go on to direct a second Bond film in

### Answer Search 만들기

In [7]:
chat_llm = ChatOpenAI()

In [8]:
ans_search_prompt_template = PromptTemplate.from_template("{context}\n질문: {question}")

In [9]:
answer_serach_chain = ans_search_prompt_template | chat_llm | StrOutputParser()

In [10]:
answer_serach_chain.invoke({"context": "철수는 구글에서 일한다", "question": "철수의 직장"})

'구글'

In [11]:
@tool("Intermediate Answer")
def answer_search(query: str) -> str:
    """search"""
    ctx = search.run(query)
    answer = answer_serach_chain.invoke({"context": ctx, "question": query})
    return answer

In [12]:
answer_search.run("아이유 나이")

'아이유는 1993년 5월 16일에 태어났기 때문에 현재 나이는 31세입니다.'

In [13]:
tools = [answer_search]

In [14]:
# Construct the Self Ask With Search Agent
agent = create_self_ask_with_search_agent(llm, tools, prompt)

In [15]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [16]:
agent_executor.invoke(
    {"input": "현재 남자 U.S. 오픈 챔피언의 고향은 어디인가요?"}
)



> Entering new AgentExecutor chain...
 Yes.
Followup: 미국 오픈 챔피언이 누구인가요?답변: 2025년 3월 30일에 열린 텍사스 칠드런스 휴스톤 오픈에서 호주 출신의 교포 이민우가 세계 랭킹 1위인 스코티 셰플러를 단 1타 차로 이기고 우승했습니다.Intermediate answer: The current male U.S. Open champion is Kim Woo-jin, a South Korean archer.
Followup: Kim Woo-jin은 어느 도시에서 태어났나요?Kim Woo-Jin은 대한민국 경기도 부천에서 태어났습니다.So the final answer is: 부천 (Bucheon, South Korea)

> Finished chain.


{'input': '현재 남자 U.S. 오픈 챔피언의 고향은 어디인가요?',
 'output': '부천 (Bucheon, South Korea)'}